# Downloading and coverting ERA5 files

Information on the fifth generation ECMWF atmospheric reanalysis of the global climate ERA5 is found at:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

# 1) Requesting ERA5 data

a) Go to the ERA5 hourly data on single levels from 1979 to present (https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form)

b) In product type select Reanalysis

c) In Variable select Popular -> 10m u-component of wind, 10m v-component of wind, and Mean sea level pressure.

d) In this example we will prepare the data for Hurricane Arthur, from 06/27/2014 to 07/09/2014. Therefore, we will have to download 2 ERA5 files, one from 06/27/2014 to 06/30/2014 and another from 07/01/2014 to 07/09/2014. 

First, we will download the data from 06/27/2014 to 06/30/2014. To do that we just need to select the Year, Month and Day accordingly. We will work with inputs every 6 hours, so select the Time: 00:00, 06:00, 12:00, and 18:00.

e) In the Geographical area tab select “Whole available region”

f) Select the GRIB format in the Format tab.

g) Click on “Show API request” and save the code.


# 2) Downloading ERA5 data

Importing the necessary lybraries:


cfgrib is installed as:
conda install -c conda-forge cfgrib
xarray is installed as:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [1]:
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import cdsapi
import datetime
c = cdsapi.Client()

Paste the API request and start downloading:

In [2]:
days = np.arange(0,15).astype(str).tolist()

In [5]:
time = ['00:00', '02:00', '04:00','06:00', '08:00', '10:00',
        '12:00', '14:00', '16:00','18:00', '20:00', '22:00',]
        #'12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        #'18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
        #
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
            'mean_sea_level_pressure',
            #'Sea_Ice_Concentration',
        ],
        'year': '2011',
        'month': '11',
        'day': days,
        'time': time,
    },
    'download.grib')

2020-09-17 16:54:00,482 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-09-17 16:54:01,185 INFO Request is queued
2020-09-17 16:54:03,896 INFO Request is running
2020-09-17 16:54:51,252 INFO Request is completed
2020-09-17 16:54:51,252 INFO Downloading http://136.156.132.153/cache-compute-0002/cache/data8/adaptor.mars.internal-1600376042.9561675-16504-6-5262ec3d-6c6b-4902-b3bf-0ffb3499925c.grib to download.grib (998.1M)
2020-09-17 16:58:55,789 INFO Download rate 4.1M/s                                                                      


Result(content_length=1046606400,content_type=application/x-grib,location=http://136.156.132.153/cache-compute-0002/cache/data8/adaptor.mars.internal-1600376042.9561675-16504-6-5262ec3d-6c6b-4902-b3bf-0ffb3499925c.grib)

# 3) Converting the .grib file into .22 (NWS=6)

Set the path to where the .grib data was saved:

Open the .grib file:

In [46]:
path = pl.Path(r'C:\Users\tmiesse\Work\adcirc-unswan\inputs')
file = 'download.grib'
grib = xr.open_dataset(path / file, engine='cfgrib')

2020-09-18 13:10:20,928 WARNING Ignoring index file 'C:\\Users\\tmiesse\\Work\\adcirc-unswan\\inputs\\download.grib.88a63.idx' older than GRIB file
2020-09-18 13:10:22,900 INFO missing from GRIB stream: 'directionNumber'
2020-09-18 13:10:22,900 INFO missing from GRIB stream: 'frequencyNumber'
2020-09-18 13:10:23,162 INFO missing from GRIB stream: 'directionNumber'
2020-09-18 13:10:23,162 INFO missing from GRIB stream: 'frequencyNumber'
2020-09-18 13:10:23,433 INFO missing from GRIB stream: 'directionNumber'
2020-09-18 13:10:23,433 INFO missing from GRIB stream: 'frequencyNumber'


Extract the variables of interest:

In [47]:
# bounding box of domain
lat1,lat2 = 44,88.5
lon1,lon2 = -230, -85

In [5]:
lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = grib.coords['time'].values[:]
idx = np.where((lon1+360 >= lon)&(lon<=lon2+360))[0]
idy = np.where((lat1 >= lat)&(lat<=lat2))[0]

In [86]:
def force2owi(file,dx,dy,dt,ilon,ilat,start,end,swlat,swlon,header,grib,lat1,lat2,lon1,lon2):
    if file == 'fort.222':
        u = grib.data_vars['u10'].values[:]
        v = grib.data_vars['v10'].values[:]
    elif file == 'fort.221':
        prmsl = grib.data_vars['msl'].values[:]
    elif file == 'fort.225':
        ice = grib.data_vars['siconc'].values[:]
    else:
        raise TypeError('not correct forcing format')
    
    
    lat = grib.coords['latitude'].values
    lon= grib.coords['longitude'].values
    idx = np.where((lon1+360 >= lon)&(lon<=lon2+360))[0]
    idy = np.where((lat1 >= lat)&(lat<=lat2))[0]
    with open(file,'w') as fin:
        fin.write(header)
        for t in range(len(dt[:])):
            param ='iLat={:4d}iLong={:4d}DX={:6.4f}DY={:6.4f}SWLat={:8.5f}SWLon={:8.3f}DT={:12s}\n'.format(ilat,ilon,dx,dy,swlat,swlon,str(dt[t].strftime('%Y%m%d%H%M')))#,width=8,prec=5)
            fin.write(param)
            i = 0
            data = []
            d1,d2 = [],[] 
            for y in idy[:]:
                for x in idx[:]:
                    if file == 'fort.222':
                        windx = u[t,y,x]
                        windy = v[t,y,x]
                        d1.append(' {:{width}.{prec}f}'.format(windx,width=9,prec=4))
                        d2.append(' {:{width}.{prec}f}'.format(windy,width=9,prec=4))
                    elif file == 'fort.221':
                        press = prmsl[t,y,x]/100
                        data.append(' {:{width}.{prec}f}'.format(press,width=9,prec=4))
                    elif file == 'fort.225':
                        sea = ice[t,y,x]
                        if np.isnan(sea):
                            sea = -1.0
                        else:
                            sea =sea * 100
                        data.append(' {:{width}.{prec}f}'.format(sea,width=9,prec=4))
            i = 0
            if file == 'fort.222':
                for d in range(len(d1)):
                    if (i == 7) :
                        fin.write(d1[d]+'\n')
                        i=0
                    elif (d+1>=len(d1)):
                        fin.write(d2[d]+'\n')
                        i = 0
                        for d in range(len(d2)):
                            if (i == 7) :
                                fin.write(d2[d]+'\n')
                                i=0
                            elif (d+1>=len(d2)):
                                fin.write(d2[d]+'\n')
                            else:
                                fin.write(d2[d])
                                i +=1
                    else:
                        fin.write(d1[d])
                        i +=1
            else:
                for d in range(len(data)):
                    if (i == 7) :
                        fin.write(data[d]+'\n')
                        i=0
                    elif (d+1>=len(data)):
                        fin.write(data[d]+'\n')
                    else:
                        fin.write(data[d])
                        i +=1
        #fin.write('\n')
    
    print(f'amount of data {len(idx)*len(idy)} data is {len(data)}')
    return print('generated owi formatted forcings')

In [91]:
ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]
dx = lon[1]-lon[0]
dy = lat[0]-lat[1]
ilon = len(idx)
ilat = len(idy)
start = ts2[0]
end = ts2[-1]
swlat = lat1
swlon = lon1#+360
header = 'Oceanweather WIN/PRE Format                            {:12s}   {:12s}\n'.format(str(start.strftime('%Y%m%d%H')),str(end.strftime('%Y%m%d%H')))#     {}'
dt = ts2
file = 'fort.222'

C:\Users\tmiesse\AppData\Local\Continuum\miniconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


In [92]:
force2owi(file,dx,dy,dt[:],ilon,ilat,start,end,swlat,swlon,header,grib,lat1,lat2,lon1,lon2)


amount of data 279777 data is 0
generated owi formatted forcings
